Changes to v4:
- store model for inference

Potential next steps:
- dots sample frequency
- a lot of images are ambiguous, trainings data needs to be cleaned: fit lower level function and check if residual is low -> remap to lower function
- which dot type is the best?
- deployment

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import math

NUM_IMAGES = 1000

# Create data directory if it doesn't exist

x_max = 100
x = np.linspace(-x_max, x_max, 100)


def get_random_linear(x):
    a = math.tan(random.uniform(math.radians(-89.0), math.radians(89.0)))
    b = random.uniform(-x_max, x_max)
    x_shift = random.uniform(-x_max, x_max)

    print("a:", a, "b:", b)

    return a * (x + x_shift) + b


def get_random_quadratic(x):
    log_a_min = np.log(0.001)  # minimum curvature
    log_a_max = np.log(10.0)  # maximum curvature
    log_a = random.uniform(log_a_min, log_a_max)
    a = np.exp(log_a) * random.choice([-1, 1])  # random sign
    b = math.tan(random.uniform(math.radians(-89.0), math.radians(89.0)))
    c = random.uniform(-x_max, x_max)
    x_shift = random.uniform(-x_max, x_max)

    x_shifted = x + x_shift

    return a * x_shifted**2 + b * x_shifted + c


def get_random_cubic(x):
    log_a_min = np.log(0.000001)  # minimum curvature derivative
    log_a_max = np.log(0.1)  # maximum curvature derivative
    log_a = random.uniform(log_a_min, log_a_max)
    a = np.exp(log_a) * random.choice([-1, 1])  # random sign
    b = random.uniform(0.001, 0.1) * random.choice([-1, 1])
    c = math.tan(random.uniform(math.radians(-89.0), math.radians(89.0)))
    d = random.uniform(-x_max, x_max)
    x_shift = random.uniform(-x_max, x_max)

    x_shifted = x + x_shift

    return a * x_shifted**3 + b * x_shifted**2 + c * x_shifted + d


def save_plot(x, y, i, label):
    # Create figure with square aspect ratio
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.plot(x, y, "x", linewidth=4)
    ax.set_xlim([-x_max, x_max])
    ax.set_ylim([-x_max, x_max])

    ax.set_xticks([])
    ax.set_yticks([])

    folder_name = f"data/{label}"

    os.makedirs(folder_name, exist_ok=True)

    file_name = f"{folder_name}/{i}.png"
    plt.savefig(file_name, dpi=50)

    print("label:", label)
    plt.close()  # Close the figure to prevent it from being displayed
    print(f"Plot saved to {file_name}")


for i in range(NUM_IMAGES):
    y_linear = get_random_linear(x)
    save_plot(x, y_linear, i, "linear")

    y_quadratic = get_random_quadratic(x)
    save_plot(x, y_quadratic, i, "quadratic")

    y_cubic = get_random_cubic(x)
    save_plot(x, y_cubic, i, "cubic")

In [ ]:
from fastai.vision.all import (
    DataBlock,
    ImageBlock,
    CategoryBlock,
    get_image_files,
    RandomSplitter,
    parent_label,
)
from pathlib import Path


path = Path("data")

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
).dataloaders(path, bs=32)

dls.show_batch(max_n=20)

In [ ]:
from fastai.vision.all import (
    vision_learner,
    resnet18,
    error_rate,
)


learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

In [ ]:
import torch
import json
from pathlib import Path


model = learn.model
model.eval()


device = next(model.parameters()).device
dummy_input = torch.randn(1, 3, 500, 500).to(device)

traced_model = torch.jit.trace(model, dummy_input)
model_folder = "model"
os.makedirs(model_folder, exist_ok=True)
traced_model.save(f"{model_folder}/model_traced.pt")
print(f"✅ TorchScript traced model saved as '{model_folder}/model_traced.pt'")


class_names = list(dls.vocab)  # Convert CategoryMap to list
with open(f"{model_folder}/class_names.json", "w") as f:
    json.dump(class_names, f)
print(f"✅ Class names saved: {class_names}")

# Save image transforms info
transforms_info = {
    "mean": [0.485, 0.456, 0.406],  # ImageNet defaults used by FastAI
    "std": [0.229, 0.224, 0.225],
    "size": 500,  # Your actual image size
}
with open(f"{model_folder}/transforms_info.json", "w") as f:
    json.dump(transforms_info, f)
print("✅ Transform info saved")

In [ ]:
from fastai.vision.all import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12, 12), dpi=60)

In [ ]:
# Function to get all misclassified images
def get_misclassified_images(learn, dls):
    """Get all misclassified images with their predictions and true labels"""
    misclassified = []

    # Get validation dataset
    val_dl = dls.valid

    # Get predictions for all validation images
    preds, targets = learn.get_preds(dl=val_dl)
    pred_labels = preds.argmax(dim=1)

    # Get file paths for validation images
    val_files = val_dl.items

    for i, (pred_label, true_label, file_path) in enumerate(
        zip(pred_labels, targets, val_files)
    ):
        if pred_label != true_label:
            misclassified.append(
                {
                    "file_path": file_path,
                    "predicted": dls.vocab[pred_label.item()],
                    "true": dls.vocab[true_label.item()],
                    "confidence": preds[i].max().item(),
                }
            )

    return misclassified


# Get misclassified images
misclassified_images = get_misclassified_images(learn, dls)
print(f"Found {len(misclassified_images)} misclassified images:")
for img in misclassified_images:
    print(
        f"File: {img['file_path'].name}, True: {img['true']}, Predicted: {img['predicted']}, Confidence: {img['confidence']:.3f}"
    )

In [ ]:
from fastai.vision.all import PILImage


# Function to plot all misclassified images
def plot_misclassified_images(misclassified_images, max_images=20):
    """Plot all misclassified images in a grid"""
    if not misclassified_images:
        print("No misclassified images found!")
        return

    n_images = min(len(misclassified_images), max_images)

    # Calculate grid dimensions
    cols = min(4, n_images)
    rows = (n_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()

    for i, img_info in enumerate(misclassified_images[:max_images]):
        # Load and display image
        img = PILImage.create(img_info["file_path"])
        axes[i].imshow(img)

        # Set title with prediction info
        title = f"True: {img_info['true']}\nPred: {img_info['predicted']}\nConf: {img_info['confidence']:.3f}"
        axes[i].set_title(title, fontsize=10)
        axes[i].axis("off")

    # Hide unused subplots
    for i in range(n_images, len(axes)):
        axes[i].axis("off")

    plt.tight_layout()
    plt.suptitle(
        f"Misclassified Images ({len(misclassified_images)} total)", fontsize=16, y=1.02
    )
    plt.show()


# Plot the misclassified images
plot_misclassified_images(misclassified_images)